# GraphRAG Quickstart

## Prerequisites
Install 3rd party packages, not part of the Python Standard Library, to run the notebook

In [2]:
! pip install --upgrade pip
! pip install python-magic
! pip install devtools  
! pip install requests 
! pip install tqdm


In [ ]:

import getpass
import json
import time
from pathlib import Path
import requests

import magic
from tqdm import tqdm
from devtools import pprint

endpoint = "http://localhost/"

## Query

Once an indexing job is complete, the knowledge graph is ready to query. Two types of queries (global and local) are currently supported. We encourage you to try both and experience the difference in responses. Note that query response time is also correlated to the TPM quota of the Azure OpenAI model you are using.

In [6]:
# a helper function to parse out the result from a query response
def parse_query_response(
    response: requests.Response, return_context_data: bool = False
) -> requests.Response | dict[list[dict]]:
    """
    Print response['result'] value and return context data.
    """
    if response.ok:
        print(json.loads(response.text)["result"])
        if return_context_data:
            return json.loads(response.text)["context_data"]
        return response
    else:
        print(response.reason)
        print(response.content)
        return response

### Global Query 

Global queries are resource-intensive, but provide good responses to questions that require an understanding of the dataset as a whole.

In [ ]:
%%time

def global_search(
    index_name: str | list[str], query: str, community_level: int
) -> requests.Response:
    """Run a global query over the knowledge graph(s) associated with one or more indexes"""
    url = endpoint + "/query/global"
    # optional parameter: community level to query the graph at (default for global query = 1)
    request = {
        "index_name": index_name,
        "query": query,
        "community_level": community_level,
    }
    return requests.post(url, json=request, headers=headers)

# perform a global query
global_response = global_search(
    index_name=index_name,
    query="whats the top 5 bikes?",
    community_level=1,
)
global_response_data = parse_query_response(global_response, return_context_data=True)
global_response_data

Based on the available data, the top two bikes are the Touring-3000 Yellow and the Touring-3000 Blue. Here are the details:

### Touring-3000 Yellow
The Touring-3000 Yellow is a central entity in the biking community. It is designed for long-distance comfort and serves as the focal point for various biking products and accessories [Data: Reports (2)]. This bike is highly regarded for its ability to provide a comfortable ride over extended distances, making it a popular choice among long-distance cyclists.

### Touring-3000 Blue
The Touring-3000 Blue is another significant bike model. It is known for its smooth ride and high-quality components, positioning it as a premium option for biking enthusiasts [Data: Reports (1)]. This bike is favored by those who seek a top-tier biking experience with reliable performance and comfort.

Unfortunately, the dataset does not provide information on the other top bikes beyond these two models. If you need more detailed information or additional bike 

{'reports': [{'id': '2',
   'title': 'Touring-3000 Yellow and Associated Biking Products',
   'occurrence weight': 1.0,
   'content': "# Touring-3000 Yellow and Associated Biking Products\n\nThe community revolves around the Touring-3000 Yellow bike, which is central to various biking products and accessories. Key entities include the Classic Vest, HL Mountain Tire, ML Headset, Women's Tights, and Water Bottle, all of which are related to enhancing the biking experience. The relationships among these entities highlight their complementary roles in providing a comprehensive biking solution.\n\n## Touring-3000 Yellow as the central entity\n\nThe Touring-3000 Yellow bike is the central entity in this community, serving as the focal point for various biking products and accessories. This bike is designed for long-distance comfort, making it a key product for touring cyclists. Its relationships with other entities, such as the Classic Vest, HL Mountain Tire, ML Headset, Women's Tights, and 

### Local Query

Local search queries are best suited for narrow-focused questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?)

In [ ]:
%%time

def local_search(
    index_name: str | list[str], query: str, community_level: int
) -> requests.Response:
    """Run a local query over the knowledge graph(s) associated with one or more indexes"""
    url = endpoint + "/query/local"
    # optional parameter: community level to query the graph at (default for local query = 2)
    request = {
        "index_name": index_name,
        "query": query,
        "community_level": community_level,
    }
    return requests.post(url, json=request, headers=headers)


# perform a local query
local_response = local_search(
    index_name=index_name,
    query="tell me about Touring-3000",
    community_level=2,
)
local_response_data = parse_query_response(local_response, return_context_data=True)
local_response_data

Method Not Allowed
b'{"detail":"Method Not Allowed"}'
CPU times: total: 0 ns
Wall time: 1.06 s


<Response [405]>